In [242]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [243]:
# Get all school districts in the US 
lea = pd.read_csv("lea_characteristics.csv", sep=",")

# Get our banned books + districts
df_school_yr_1 = pd.read_csv("PEN_July_1_2021_June_30_2022.csv", sep=",")
df_school_yr_2 = pd.read_csv("PEN_July_1_2022_December_31_2022.csv", sep=",")
books = df_school_yr_1.append(df_school_yr_2)

In [244]:
# Create a new binary variable that is "book bans" and set true given our "data" df
books['district_name'] = books['District'].str.lower()
lea['district_name'] = lea['LEA_NAME'].str.lower()
books["STATE"] = books["State"].str.upper()

# Strip of common words
common_words = ["city school district", "schools", "school", "district", "public", "county", 
                "independent", "cisd", "isd", " sd", " co ", "pblc", "schs", "high", "of", 
                "community", "comm", "area", "the", "department", "consolidated", "  no",
                "r-iii", "r-vii", "r-ix", "r-iv", "r-1", "r-i", 
                "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "#", "-", "."]

for word in common_words: 
    books['district_name'] = books['district_name'].str.replace(word, " ").str.strip()
    lea['district_name'] = lea['district_name'].str.replace(word, " ").str.strip()

In [245]:
# make sure they are unique
pairs = {}
for district in pd.unique(books["district_name"]): 
    pen_names = books[books["district_name"]==district][["STATE", "District"]]
    lea_names = lea[lea["district_name"]==district][["LEA_STATE_NAME", "LEA_NAME"]]
    
    for pen_name in pd.unique(pen_names["District"]): 
        state = pen_names[pen_names["District"] == pen_name]["STATE"].values[0]
        lea_name = lea_names[lea_names["LEA_STATE_NAME"] == state]["LEA_NAME"].values
        if len(lea_name) > 0: 
            lea_name = lea_name[0]
            pairs[pen_name] = lea_name

outliers_pen_to_lea = {
    "Medford School District 549C": "Medford SD 549C", 
    "Virginia Beach City Public Schools": "VA BEACH CITY PBLC SCHS",
    "Harlem School District 122": "Harlem UD 122", 
    "Regional School Unit No. 56": "RSU 56", 
    "ROWVA CUSD 208": "R O W V A CUSD 208", 
    "Cherry Creek School District": "Cherry Creek School District No. 5 in the County of Arapaho", 
    "Raytown Quality Schools": "RAYTOWN C-2", 
    "Williston Basin School District #7": "WILLISTON 1", 
    "Lexington-Richland School District 5": "School District 5 of Lexington and Richland Counties", 
    "Madison County Schools": "MADISON CO SCHOOL DIST", 
}

matching_district_names = pairs | outliers_pen_to_lea

books['matched_name'] = [matching_district_names.get(d, None) for d in books['District']]

In [246]:
# Only keep specific cols of a df so that we can start fresh with a LEAIID
cols_to_keep = ["LEA_STATE_NAME", "LEAID", "LEA_NAME", "LEA_ENR"]
lea = lea[cols_to_keep]
merged = lea.copy()
merged["banned_books"] = np.where((lea['LEA_NAME'].isin(books['matched_name'])) & (lea['LEA_STATE_NAME'].isin(books['STATE'])), True, False)
merged = merged.loc[merged["LEA_STATE_NAME"] == "TEXAS"]


In [247]:
### TLDR; I'm dumb and didn't realize there was an LEA_ENR col. But verified that both ways of caluclating achieves the same result!
'''
# Add enrollment to our table
enrollment = pd.read_csv("Enrollment.csv")
enrollment["Total_enrollment"] = enrollment["TOT_ENR_M"] + enrollment["TOT_ENR_F"]
cols = ["LEAID", "Total_enrollment"]
enrollment = enrollment[cols]
enrollment = enrollment.groupby('LEAID')["Total_enrollment"].sum().reset_index() # Sum enrollment across all schools in a district

# Merge with our merged df
# Cast our LEAID to a numeric or else the merge doesn't work correctly
merged['LEAID'] = pd.to_numeric(merged['LEAID'], errors='coerce')
enrollment['LEAID'] = pd.to_numeric(enrollment['LEAID'], errors='coerce')
merged = pd.merge(merged, enrollment, on='LEAID', how='left')

merged
'''

'\n# Add enrollment to our table\nenrollment = pd.read_csv("Enrollment.csv")\nenrollment["Total_enrollment"] = enrollment["TOT_ENR_M"] + enrollment["TOT_ENR_F"]\ncols = ["LEAID", "Total_enrollment"]\nenrollment = enrollment[cols]\nenrollment = enrollment.groupby(\'LEAID\')["Total_enrollment"].sum().reset_index() # Sum enrollment across all schools in a district\n\n# Merge with our merged df\n# Cast our LEAID to a numeric or else the merge doesn\'t work correctly\nmerged[\'LEAID\'] = pd.to_numeric(merged[\'LEAID\'], errors=\'coerce\')\nenrollment[\'LEAID\'] = pd.to_numeric(enrollment[\'LEAID\'], errors=\'coerce\')\nmerged = pd.merge(merged, enrollment, on=\'LEAID\', how=\'left\')\n\nmerged\n'

In [248]:
# Calculate teacher to student ratio
teacher_student_ratio = pd.read_csv("School_Support.csv", encoding='latin-1')
cols = ["LEAID", "SCH_FTETEACH_TOT"]

teacher_student_ratio = teacher_student_ratio[cols]
teacher_student_ratio = teacher_student_ratio.groupby('LEAID')["SCH_FTETEACH_TOT"].sum().reset_index() # Sum enrollment across all schools in a district
# Cast our LEAID to numeric
merged['LEAID'] = pd.to_numeric(merged['LEAID'], errors='coerce')
teacher_student_ratio['LEAID'] = pd.to_numeric(teacher_student_ratio['LEAID'], errors='coerce')
merged = pd.merge(merged, teacher_student_ratio, on='LEAID', how='left')

merged["SCH_FTETEACH_TOT"] = merged["SCH_FTETEACH_TOT"] / merged["LEA_ENR"]

In [303]:
# Look at school expenditures
moolah = pd.read_csv("School Expenditures.csv", encoding='latin-1')
moolah

,LEA_STATE,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,COMBOKEY,JJ,SCH_SAL_TOTPERS_WOFED,SCH_FTE_TEACH_WOFED,...,SCH_SAL_ADM_WOFED,SCH_SAL_TEACH_WFED,SCH_FTE_AID_WFED,SCH_SAL_AID_WFED,SCH_FTE_SUP_WFED,SCH_SAL_SUP_WFED,SCH_FTE_ADM_WFED,SCH_SAL_ADM_WFED,SCH_SAL_TOTPERS_WFED,SCH_NPE_WFED
0,AL,ALABAMA,0100002,Alabama Youth Services,1705,Wallace Sch - Mt Meigs Campus,010000201705,Yes,1908676.0,25.00,...,81533.0,1827143.00,0.0,0.0,0.00,0.00,1.0,81533.0,1908676.00,102500.00
1,AL,ALABAMA,0100002,Alabama Youth Services,1706,McNeel Sch - Vacca Campus,010000201706,Yes,1563570.0,16.00,...,82000.0,1223463.00,0.0,0.0,0.00,0.00,0.0,82000.0,1305463.00,0.00
2,AL,ALABAMA,0100002,Alabama Youth Services,1876,Alabama Youth Services,010000201876,No,3998276.0,60.00,...,81533.0,3400850.00,0.0,0.0,0.00,0.00,1.0,81533.0,5122976.00,0.00
3,AL,ALABAMA,0100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,010000299995,Yes,305220.0,3.00,...,81553.0,305220.00,0.0,0.0,0.00,0.00,1.0,82000.0,386551.00,18700.00
4,AL,ALABAMA,0100005,Albertville City,870,Albertville Middle School,010000500870,No,2394774.5,38.29,...,257464.0,1944222.84,3.0,66178.0,13.89,514793.08,3.0,257464.0,2782657.92,1059055.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97627,PR,PUERTO RICO,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,2079,SEVERO E. COLBERG RAMIREZ,720003002079,No,0.0,0.00,...,-9.0,1373160.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00
97628,PR,PUERTO RICO,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,2082,INTERMEDIA JUAN SERRALLES,720003002082,No,0.0,0.00,...,-9.0,676944.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00
97629,PR,PUERTO RICO,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,2083,MARTIN GONZALEZ,720003002083,No,0.0,0.00,...,-9.0,393570.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00
97630,PR,PUERTO RICO,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,2084,NEMESIO R. CANALES I,720003002084,No,0.0,0.00,...,-9.0,638940.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00


In [249]:
# Retention table looks quite... sparse
retention = pd.read_csv("Retention.csv", encoding='latin-1')
retention = retention.loc[retention["LEA_STATE"] == "TX"]
retention

,LEA_STATE,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,COMBOKEY,JJ,SCH_RET_KG_IND,SCH_RET_G01_IND,...,SCH_RET_G12_TR_M,SCH_RET_G12_TR_F,TOT_RET_G12_M,TOT_RET_G12_F,SCH_RET_G12_LEP_M,SCH_RET_G12_LEP_F,SCH_RET_G12_IDEA_M,SCH_RET_G12_IDEA_F,SCH_RET_G12_504_M,SCH_RET_G12_504_F
78831,TX,TEXAS,4800001,CROSBYTON CISD,1145,CROSBYTON EL,480000101145,No,Yes,Yes,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
78832,TX,TEXAS,4800001,CROSBYTON CISD,1146,CROSBYTON SECONDARY,480000101146,No,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
78833,TX,TEXAS,4800001,CROSBYTON CISD,3621,SP ED CO-OP,480000103621,No,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
78834,TX,TEXAS,4800002,SPUR ISD,4732,SPUR SCHOOL,480000204732,No,No,No,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
78835,TX,TEXAS,4800003,ROCKSPRINGS ISD,4219,ROCKSPRINGS K-12,480000304219,No,No,Yes,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87584,TX,TEXAS,4846740,ZAVALLA ISD,5353,ZAVALLA EL,484674005353,No,Yes,Yes,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
87585,TX,TEXAS,4846740,ZAVALLA ISD,5354,ZAVALLA H S,484674005354,No,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
87586,TX,TEXAS,4846770,ZEPHYR ISD,5356,ZEPHYR SCHOOL,484677005356,No,Yes,No,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
87587,TX,TEXAS,4899130,BENAVIDES ISD,5627,BENAVIDES SECONDARY,489913005627,No,-9,-9,...,0,0,1,0,0,0,0,0,0,0


In [250]:
# Merge in county information
county_info = pd.read_excel("county_info.xlsx")

In [251]:
# Add in county information for all LEAIDs
cols = ["LEAID", "NMCNTY"]
county_info = county_info[cols]
county_info['LEAID'] = pd.to_numeric(county_info['LEAID'], errors='coerce')

# Get the most common county for each school district
county_info = county_info.groupby('LEAID')['NMCNTY'].agg(lambda x: x.mode().iloc[0])

merged = pd.merge(merged, county_info, on='LEAID', how='left')

In [252]:
merged = merged.dropna() # only 9 instances of NA

# Calculate the Political Leaning per county
tpi = pd.read_csv("TPI.csv")
pattern1 = r'(?P<Prefix1>[A-Za-z]+)-(?P<Percentage1>\d+)%' # REGEX for the pattern
pattern2 = r'(?P<Prefix2>[A-Za-z]+)-(?P<Percentage2>\d+)%' # REGEX for the pattern

df_extracted1 = tpi['Current TPI (20-22)'].str.extract(pattern1)
df_extracted2 = tpi['Previous TPI (18-20)'].str.extract(pattern2)

# Concatenate the extracted columns with the original DataFrame
df_result = pd.concat([tpi, df_extracted1, df_extracted2], axis=1)
df_result["Prefix1"] = np.where(df_result["Prefix1"] == "R", 1, -1).astype(float)
df_result["Prefix2"] = np.where(df_result["Prefix2"] == "R", 1, -1).astype(float)

# Case our columns to perform our calculation
df_result["Percentage1"] = df_result["Percentage1"].astype(float)
df_result["Percentage2"] = df_result["Percentage2"].astype(float)

df_result["overall_TPI"] = ((df_result["Prefix1"] * df_result["Percentage1"] * 0.01) + (df_result["Prefix2"] * df_result["Percentage2"] * 0.01)) / 2

In [286]:
merged.sort_values(by="NMCNTY")

,LEA_STATE_NAME,LEAID,LEA_NAME,LEA_ENR,banned_books,SCH_FTETEACH_TOT,NMCNTY
334,TEXAS,4813200,CAYUGA ISD,576,False,0.085938,anderson
1039,TEXAS,4840500,SLOCUM ISD,423,False,0.131560,anderson
877,TEXAS,4834050,PALESTINE ISD,3419,False,0.075168,anderson
835,TEXAS,4832250,NECHES ISD,360,False,0.093889,anderson
1113,TEXAS,4843290,WESTWOOD ISD,1564,False,0.095358,anderson
...,...,...,...,...,...,...,...
846,TEXAS,4832700,NEWCASTLE ISD,231,False,0.082251,young
549,TEXAS,4821360,GRAHAM ISD,2373,False,0.069954,young
1198,TEXAS,4846710,ZAPATA COUNTY ISD,3521,False,0.062482,zapata
688,TEXAS,4826250,LA PRYOR ISD,468,False,0.091880,zavala


In [297]:
# Merge our books df with our county leaning df
merged["NMCNTY"] = merged["NMCNTY"].str.lower()
merged["NMCNTY"] = merged["NMCNTY"].str.replace('county', '', case=False).str.strip()
df_result["County Name"] = df_result["County Name"].str.lower().str.strip()
df_result = df_result[["County Name", "overall_TPI"]]
df_result["NMCNTY"] = df_result["County Name"].astype(str)
merged["NMCNTY"] = merged["NMCNTY"].astype(str)

merged = merged.merge(df_result, on='NMCNTY', how='left')

In [300]:
merged.dropna()

,LEA_STATE_NAME,LEAID,LEA_NAME,LEA_ENR,banned_books,SCH_FTETEACH_TOT,NMCNTY,County Name,overall_TPI
0,TEXAS,4800001,CROSBYTON CISD,365,False,0.101370,crosby,crosby,0.740
1,TEXAS,4800002,SPUR ISD,261,False,0.107280,dickens,dickens,0.865
2,TEXAS,4800003,ROCKSPRINGS ISD,313,False,0.076677,edwards,edwards,0.850
3,TEXAS,4800004,POR VIDA ACADEMY,273,False,0.091575,bexar,bexar,-0.570
4,TEXAS,4800005,BLACKWELL CISD,155,False,0.129032,mitchell,mitchell,0.855
...,...,...,...,...,...,...,...,...,...
1188,TEXAS,4846680,YSLETA ISD,41198,False,0.066469,el paso,el paso,-0.685
1189,TEXAS,4846710,ZAPATA COUNTY ISD,3521,False,0.062482,zapata,zapata,-0.585
1190,TEXAS,4846740,ZAVALLA ISD,368,False,0.092391,angelina,angelina,0.750
1191,TEXAS,4846770,ZEPHYR ISD,202,False,0.103960,brown,brown,0.875


In [301]:
merged.to_csv("Final_DF.csv")

In [278]:
df_result

,County Name,overall_TPI,NMCNTY
0,roberts,0.965,roberts
1,king,0.965,king
2,borden,0.955,borden
3,glasscock,0.945,glasscock
4,motley,0.935,motley
...,...,...,...
249,el paso,-0.685,el paso
250,jim hogg,-0.695,jim hogg
251,zavala,-0.725,zavala
252,presidio,-0.705,presidio


In [254]:
# Instead of merging, create a binary variable where a column exists
lea["banned_books"] = np.where((lea['LEA_NAME'].isin(books['matched_name'])) & (lea['LEA_STATE_NAME'].isin(books['STATE'])), True, False)

# print(lea.loc[lea["banned_books"] == True])

In [255]:
# Now we can merge in additional dataframes of interest

'''
Dropout Count (TOTD912)
Enrollment Base (EBS912)
Dropout Rate (DRP912)
Diploma Counts (TOTDPL)
AFGR Enrollment Base Counts (AFGEB)
'''

# Need a better/more recent dataset

lea_academic_success = pd.read_csv("lea_academic_success.txt", delim_whitespace=True)
lea_academic_success.drop(columns=["SURVYEAR", "FIPST"], inplace=True)

In [256]:
# Merge our new academic success DF with our other data on LEAID
final_df = lea.merge(lea_academic_success, on='LEAID', how='left', indicator=True)

final_df.to_csv("test1.csv")

In [257]:
lea["LEA_STATE"]

KeyError: 'LEA_STATE'

In [ ]:
print(lea.loc[(lea["LEA_STATE"] == "TX") & (lea["banned_books"] == True)])

KeyError: 'LEA_STATE'

In [ ]:
print(lea.loc[(lea["LEA_STATE"] == "TX")])

print(lea.loc[(lea["LEA_STATE"] == "TX")])

      LEA_STATE LEA_STATE_NAME    LEAID           LEA_NAME        LEA_ADDRESS  \
15027        TX          TEXAS  4800001     CROSBYTON CISD  204 S HARRISON ST   
15028        TX          TEXAS  4800002           SPUR ISD        P O BOX 550   
15029        TX          TEXAS  4800003    ROCKSPRINGS ISD        P O BOX 157   
15030        TX          TEXAS  4800004   POR VIDA ACADEMY    1135 MISSION RD   
15031        TX          TEXAS  4800005     BLACKWELL CISD        P O BOX 505   
...         ...            ...      ...                ...                ...   
16224        TX          TEXAS  4846680         YSLETA ISD       9600 SIMS DR   
16225        TX          TEXAS  4846710  ZAPATA COUNTY ISD        P O BOX 158   
16226        TX          TEXAS  4846740        ZAVALLA ISD      431 E MAIN ST   
16227        TX          TEXAS  4846770         ZEPHYR ISD       11625 CR 281   
16228        TX          TEXAS  4899130      BENAVIDES ISD        PO DRAWER P   

          LEA_CITY LEA_ZIP 